<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Deep Learning
Your first model took random shots and your second model tried matching the current game against past games; however, if no past games matched then it just randomly guessed. This model will do better by __learning__ the pattern of ship layouts, not just memorizing them. 

## Data
We will be using past game data to train our model. First, we need to download the data from the master game archive by running: (note the data is constantly being updated as games are played)
You will be executed the following commands in the `Code` section below. 
```bash
# Download, parse data, create sample data
./bin/download_data.sh
# Create directory to store the downloaded data
mkdir ./containers/Shoot/CNN/mock/data/
# Next copy the sample data file into the mock directory of the code for this model 
cp ./data/data.min.json ./containers/Shoot/CNN/mock/data/data.json
```
You will now be able to test your model training localy

In [ ]:
%cd /home/ec2-user/SageMaker/GameDayRepo

In [ ]:
!pwd;./bin/download_data.sh;mkdir -p ./containers/Shoot/CNN/mock/data/;cp ./data/data.min.json ./containers/Shoot/CNN/mock/data/data.json

## Code
Please confirm that the Operations team has completed the initial setup including registration of the team name. Also, check with the instructor if the GameEngine has started. 
Please execute the following command from the terminal.
Code is in /containers/Shoot/CNN contains the following files
    
- ### ./containers/Shoot/CNN/Makefile
Makefile for creating the tar.gz file of code for [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) framework 

- ### [./containers/Shoot/CNN/host.py](/edit/GameDayRepo/containers/Shoot/CNN/host.py)
The code for generating inferences (don't run this file directly). In addition to running our model, it will also put a tag in the session attribute that we will use in Athena to compare different models. You can change the tag value (which you should do for each model) by editing the host.py file.

- ### [./containers/Shoot/CNN/test.py](/edit/GameDayRepo/containers/Shoot/CNN/test.py)
code for testing the host.py and train.py files locally. It creates a mock [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) runtime in order to run train.py and then test.py. You can edit this file in order to customize your test.  If you get a message about missing dependencies (like mxnet), just install them with:
```shell
pip install mxnet # for example
```

- ### [./containers/Shoot/CNN/train.py](/edit/GameDayRepo/containers/Shoot/CNN/train.py)
the training code (don't run this file directly)


Next, run the test script locally to confirm the scripts works. To do this, you must download the dataset and create a sample data set. Do this by running the download script in the bin directory. 
```shell
./bin/download_data.sh #downloads data from master Archive, parses it, and also create data sample
```

Then in the code directory (/containers/Shoot/CNN) run:
```shell
mkdir -p mock/data 
cp ../../../data/data.min.json mock/data/data.json #copy the data sample to the mock directory
```
Now you can run the test script:
```shell
python ./test.py
```

In [ ]:
!cd ./containers/Shoot/CNN/;python test.py

## Configure
Before you can deploy your second model, you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit the deployment configuration in [/shoot-config.json](/edit/GameDayRepo/shoot-config.json). You need to edit the following field to the following values

|parameter field|Value|description|
|---|---|---|
|trainsourcefile|`shoot-CNN.tar.gz`| code for our model|
|hostsourcefile|`shoot-CNN.tar.gz`|a code for our model|
|hyperparameters.epochs|3|Max epochs to train for|
|hyperparameters.warm_up|1|How many epochs to wait to start early stoppping|
|hyperparameters.patience|1|How many epochs with no improvement before early stopping|
|hyperparameters.learning_rate|0.005||
|hyperparameters.width|32|How many feature filters in each layer|
|hyperparameters.depth|4|how many layers|
|hyperparameters.batch_size|10|batch size for training|
|metrics|[{Name:"Validation",Regex:"Testing loss: (.*?);"},{Name:"Training",Regex:"Training loss: (.*?);"},{Name:"Throughput",Regex:"Throughput=(.*?);"}]|Metrics to send to cloudwatch logs|
|traininstancetype|"ml.m5.xlarge"|instance type to train with|
|channels|{"train":{"path":"all/data.json"}}|path to data in data bucket|

In [ ]:
!cp shoot-config-cnn.json shoot-config.json

## Deploy
1. Follow instructions from endpoint reference to commit your code and then push your changes to the remote branch.
2. Download, parse and upload the training data by running the download_data.sh and upload_data.sh scripts
```shell
./bin/download_data.sh
./bin/upload_data.sh
```


In [ ]:
!./bin/download_data.sh
!./bin/upload_data.sh

3. Tell Operations to deploy your changes

## Training Evaluation
After your model has run, there are two ways you will want to evaluate your model
    1. Validation Performance: This is how well your model did on the hold out data set. 
    2. Training Throughput: How many data items can it train on per second.

To view these metrics:
1. Go to the [sagemaker training jobs](https://us-west-2.console.aws.amazon.com/sagemaker/home?#/jobs)
1. Select the training job you want to evaluate (probably the most recent one)
<img src="../assets/sagemaker-training-jobs.jpg" width=500></img>
2. Scroll down till you see the monitor card. You will see three options
<img src="../assets/sagemaker-monitor.jpg" width=500></img>
    1. View Algorithm Metrics: will open CloudWatch and you can view a plot of the metrics of your model. For this model, you can view the training score, validation score, and throughput. 
    2. View Logs: Will take you to CloudWatch logs to see the log output of the models container (useful for debugging).
    3. View Instance Metrics: will open CloudWatch and you can view a plot of the metrics (CPU utilization, GPU utilization, Memory usage, etc...)
    
### Validation Performance
The model you trained uses the [DICE score](https://forums.fast.ai/t/understanding-the-dice-coefficient/5838) which will be a number between 0.0 and 1.0 with 1.0 representing a perfect model. You want your model to have a high dice score as much as possible. Increase the Width and Depth hyperparameters will make your model more powerful and give you a higher dice score (but will take longer to train)

### Training Throughput
Your first training job should have trained for just a few epochs. This will give you a good estimate of the training throughput which you can then use to adjust how many epochs to train your next model. Too low and your model does not learn enough, too long and your model never finished in time to gain you points. As a team you will need to decided how long you want to train a model for. Note, increasing the width and depth hyperparamters will decrease your throughput but using a larger instance type will increase the throughput. 

## Deployment Performance
Next, monitor you models performance. See [Athena](../Athena.ipynb) for details on setting up Athena (replace values between `<>` like in the Athena instructions). The following query will show win percentage by shooting type. 

```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.shootType') AS type,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  type
```

Try editing hyperparamters or the code to improve your performance results. 